In [1]:
import pandas as pd
import numpy as np
from scipy.stats import mode
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold

In [2]:
data_path = 'data.csv'
shops_data = pd.read_csv(data_path, header = None) #rows: 349586 columns: 71

In [3]:
def getChoice(data, n):
    perfmat = np.empty([10,7])
    
    for i in range(0,7):
        for j in range(0,10):
            perfmat[j,i] = data.iloc[n, ((i)*10 + j)]
    return perfmat

def printChoice(data, n):
    arr = getChoice(data, n)
    row_names = ['Albert Heijn',
                        'Aldi',
                        'C1000',
                        'Dekamarkt',
                        'Edah',
                        'Jumbo',
                        'Konmar',
                        'Lidl',
                        'Plus',
                        'Super de Boer']
    col_names = ['Dist',
                         'Floor',
                         'Loyalty',
                         'Price',
                         'PriceImage',
                         'Feature',
                         'Lag']
    df = pd.DataFrame(data=arr, index=row_names, columns=col_names)
    print(df)

In [4]:
shops_data.head()

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,0.5524,4.8413,5.5765,62.543,2.6894,4.1679,10.294,1.0149,10.397,0.9884,...,0,0,0,0,0,0,0,0,1,10
1,0.5524,4.8413,5.5765,62.543,2.6894,4.1679,10.294,1.0149,10.397,0.9884,...,0,0,0,0,0,0,0,0,1,10
2,0.5524,4.8413,5.5765,62.543,2.6894,4.1679,10.294,1.0149,10.397,0.9884,...,0,0,0,0,0,0,0,0,1,10
3,0.5524,4.8413,5.5765,62.543,2.6894,4.1679,10.294,1.0149,10.397,0.9884,...,0,0,0,0,0,0,0,0,1,10
4,0.5524,4.8413,5.5765,62.543,2.6894,4.1679,10.294,1.0149,10.397,0.9884,...,0,0,0,0,0,0,0,0,1,10


In [14]:
def processChoice(arr, client_number):
    #ZMIENIAM NUMERACJĘ Z 1-10 NA 0-9
    choice = int(shops_data.iloc[client_number][70] - 1)
    compared_rows = []
    for i in range(0,10):
        if i != choice:
            tmp_list = []
            #DISTANCE
            tmp_list.append(arr[i][0] - arr[choice][0])
            #FLOOR
            tmp_list.append(arr[i][1] - arr[choice][1])
            #LOYALITY
            tmp_list.append(arr[choice][2] - arr[i][2])
            #PRICE
            tmp_list.append(arr[i][3] - arr[choice][3])
            #PRICEImage
            tmp_list.append(arr[i][4] - arr[choice][4])
            #Feature
            tmp_list.append(arr[i][5] - arr[choice][5])
            #LAG
            tmp_list.append(arr[choice][6] - arr[i][6])
            #CHOICE
            tmp_list.append(choice)
            compared_rows.append(tmp_list)
    return compared_rows
            

In [13]:
for i in range(349584,349586):
    printChoice(shops_data,i)

                   Dist    Floor  Loyalty   Price  PriceImage  Feature  Lag
Albert Heijn    0.90808  1.50000      0.0  5.3855     1.40430  2.00610  0.0
Aldi            0.67574  0.43244      0.0  6.8136     0.75628  0.87002  0.0
C1000           0.67574  1.15000      0.0  6.1502     0.90220  0.57712  0.0
Dekamarkt      12.74700  0.68400      0.0  6.0817     0.93125  1.65120  0.0
Edah            4.97350  0.88700      0.0  5.8466     1.01230  2.72110  0.0
Jumbo           0.86777  0.75200      0.0  8.0000     0.93776  0.00000  1.0
Konmar         38.08800  2.92500      0.0  5.4171     0.98980  1.32670  0.0
Lidl            0.89923  1.10000      0.0  6.7108     0.74814  9.10080  0.0
Plus            6.44040  0.92000      0.0  5.8279     1.15740  2.74800  0.0
Super de Boer   3.14780  0.81000      1.0  7.6000     1.02720  3.66250  0.0
                   Dist    Floor  Loyalty   Price  PriceImage   Feature  Lag
Albert Heijn    0.90808  1.50000      0.0  5.3855     1.27550  0.203770  0.0
Aldi      

In [15]:
test_data = shops_data
processed_df = pd.DataFrame(processChoice(getChoice(shops_data,0),0))
for i in range(1, 349586): #349585
    processed_df = processed_df.append(pd.DataFrame(processChoice(getChoice(shops_data,i),i)), ignore_index=True)
processed_df.columns = ['DIS','FLO','LOY','PRI','PIM','FEA','LAG','CHO']
processed_df
processed_df.to_csv('procDF2.csv', index=False)  

In [25]:
shops_features = ['DIS','FLO','LOY','PRI','PIM','FEA','LAG']
X = processed_df[shops_features]
y = processed_df.CHO
y.nunique()

10